# LLM-Based D&D Simulation Tutorial

This notebook demonstrates how to use the LLM-based D&D simulation system to create AI-driven gameplay sessions based on real human campaign data.

## Overview

The simulation system includes:
- **Campaign Parameter Extraction**: Analyze human campaigns to extract initialization parameters
- **Character Creation**: Generate D&D characters with personalities and classes
- **Turn Sampling**: Create realistic turn sequences using various sampling methods
- **Character Agents**: LLM-powered characters with memory and decision-making
- **Game Session Management**: Orchestrate complete D&D scenarios

## Prerequisites

1. Install required packages: `pip install anthropic`
2. Set your Anthropic API key: `export ANTHROPIC_API_KEY=your_key_here`
3. Ensure you have access to the human campaign data files

## Setup and Imports

In [1]:
import os
import sys
import json
import random
from pathlib import Path

# Add the llm_scaffolding directory to the Python path
sys.path.append(str(Path.cwd().parent / "llm_scaffolding"))

# Import our simulation system
from dnd_simulation import (
    extract_campaign_parameters,
    create_characters,
    sample_turn_sequence,
    CharacterAgent,
    GameSession
)

# Set random seed for reproducible results
random.seed(42)

print("✅ Setup complete!")

✅ Setup complete!


## API Key Setup (Secure Method)

We'll load the API key from a secure file that's excluded from version control.

**First-time setup:**
1. Open the `api_key.txt` file in the project root directory
2. Replace the placeholder text with your actual Anthropic API key
3. Save the file (it's already in .gitignore so it won't be committed)

**Alternative methods:**
- Environment variable: `export ANTHROPIC_API_KEY=your_key_here`
- Direct assignment (not recommended): `api_key = 'your_key_here'`

In [2]:
def load_api_key():
    """Load API key from secure file or environment variable."""

    # Method 1: Try to load from secure file
    api_key_file = Path.cwd().parent / "api_key.txt"

    if api_key_file.exists():
        with open(api_key_file, 'r') as f:
            content = f.read().strip()

            # Skip comment lines and empty lines
            lines = [
                line.strip() for line in content.split('\n')
                if line.strip() and not line.strip().startswith('#')
            ]

            if lines and lines[0] != 'PASTE_YOUR_ANTHROPIC_API_KEY_HERE':
                api_key = lines[0]
                print("✅ API key loaded from api_key.txt")
                print(f"Key prefix: {api_key[:10]}...")
                return api_key
            else:
                print(
                    "⚠️  api_key.txt found but contains placeholder text")
                print(
                    "Please edit api_key.txt and paste your actual API key"
                )

    # Method 2: Try environment variable
    api_key = os.getenv('ANTHROPIC_API_KEY')
    if api_key:
        print("✅ API key loaded from environment variable")
        print(f"Key prefix: {api_key[:10]}...")
        return api_key

    return None

# Load the API key
api_key = load_api_key()
os.environ['ANTHROPIC_API_KEY'] = api_key 

✅ API key loaded from api_key.txt
Key prefix: sk-ant-api...


## Step 1: Extract Campaign Parameters

Let's load a real human campaign and extract parameters that we can use to initialize our simulation.

In [3]:
# Choose a campaign file to analyze
campaign_file = "../data/raw-human-games/individual_campaigns/10391-guardians-of-gridori.json"

# Extract parameters from the human campaign
print("📊 Extracting campaign parameters...")
campaign_params = extract_campaign_parameters(campaign_file)

print(f"\n🎯 Campaign Parameters:")
print(f"  Campaign Name: {campaign_params['campaign_name']}")
print(f"  Total Messages: {campaign_params['total_messages']:,}")
print(f"  Number of Players: {campaign_params['num_players']}")
print(f"  Character Names: {campaign_params['character_names']}")
print(f"  Player Names: {campaign_params['player_names']}")
print(f"  Character Classes: {campaign_params['character_classes']}")
print(f"  Character Genders: {campaign_params['character_genders']}")
print(f"  Character Races: {campaign_params['character_races']}")
print(f"  Character personalities: {campaign_params['character_personalities'][0]}")
print(f"  Initial Scenario: {campaign_params['initial_scenario']}")


📊 Extracting campaign parameters...

🎯 Campaign Parameters:
  Campaign Name: 10391-guardians-of-gridori
  Total Messages: 216
  Number of Players: 5
  Character Names: ['Dungeon Master' 'Jinx' 'Faen' 'Dmitrei' 'Thokk']
  Player Names: ['LightSpeed', 'AmazingAmazon', 'ShyVideoGamer', 'JacWalke', 'Finny']
  Character Classes: [None, 'druid', 'Warlock', 'Druid', 'Barbarian']
  Character Genders: [None, 'female', 'female', 'male', 'male']
  Character Races: [None, 'Dwarf', 'elf', 'Human', 'Half-Orc']
  Character personalities: The Dungeon Master (DM) takes a measured and patient approach, carefully guiding the story and providing detail-rich descriptions to immerse the players. They seem adept at balancing player agency with narrative progression, allowing the characters to explore and make choices while still maintaining a cohesive world and plot.
  Initial Scenario: {'1': {'date': '2017-12-16T13:43:31', 'player': 'LightSpeed', 'character': 'Dungeon Master', 'in_combat': False, 'paragraph

## Step 2: Create Characters

Now let's create our AI characters using the player count from the human campaign.

In [4]:
# Create characters based on the campaign parameters
print("🧙 Creating characters...")

# For now, we'll use default characters (future versions can extract from campaign)
characters = create_characters(campaign_params)

print(f"\n✨ Created {len(characters)} characters:")
for char in characters:
    print(f"  🗡️  {char.name} ({char.dnd_class}): {char.personality}")

# Get character names for turn sampling
character_names = [char.name for char in characters]
print(f"\n👥 Character roster: {character_names}")

🧙 Creating characters...

✨ Created 5 characters:
  🗡️  Dungeon Master (None): The Dungeon Master (DM) takes a measured and patient approach, carefully guiding the story and providing detail-rich descriptions to immerse the players. They seem adept at balancing player agency with narrative progression, allowing the characters to explore and make choices while still maintaining a cohesive world and plot.
  🗡️  Jinx (druid): Jinx, the dwarven druid, appears to be a steadfast and earnest player who is deeply committed to their character's beliefs and goals. They demonstrate a strong sense of duty and protectiveness towards the party, but can also be a bit reckless at times, rushing headlong into danger. Their playstyle suggests a player who values roleplaying and character development over pure combat optimization.
  🗡️  Faen (Warlock): Faen, the elven warlock, seems to be a more reserved and introspective player, often letting the other characters take the lead. However, their character'

## Step 3: Generate Turn Sequence

Create a realistic turn sequence using uniform sampling across all characters.

In [5]:
# Generate a turn sequence for our mini-session
total_turns = 5  # Adjust for desired session length

print(f"🎲 Generating turn sequence ({total_turns} turns)...")
turn_sequence = sample_turn_sequence(
    character_names=character_names,
    total_turns=total_turns,
    method='uniform'
)

print(f"\n📋 Turn sequence: {turn_sequence}")

# Show turn distribution
from collections import Counter
turn_counts = Counter(turn_sequence)
print(f"\n📊 Turn distribution:")
for char_name, count in turn_counts.items():
    print(f"  {char_name}: {count} turns ({count/total_turns*100:.1f}%)")

🎲 Generating turn sequence (5 turns)...

📋 Turn sequence: ['Dmitrei', 'Dungeon Master', 'Jinx', 'Jinx', 'Dmitrei']

📊 Turn distribution:
  Dmitrei: 2 turns (40.0%)
  Dungeon Master: 1 turns (20.0%)
  Jinx: 2 turns (40.0%)


## Step 4: Initialize Game Session

Create the game session manager that will orchestrate our D&D simulation.

In [6]:
# Create game session
print("🎭 Initializing game session...")
game_session = GameSession(characters=characters, api_key=api_key)

print(f"✅ Game session created with {len(characters)} characters")
print(f"📝 Game log initialized (currently {len(game_session.game_log)} events)")

🎭 Initializing game session...
✅ Game session created with 5 characters
📝 Game log initialized (currently 0 events)


## Step 6: Run the Simulation

Now let's run the complete D&D simulation and watch our AI characters interact!

In [ ]:
# Run the simulation
print("🚀 Starting D&D simulation...")
print("\n" + "="*60)

# Execute the scenario
game_session.run_scenario(initial_scenario=campaign_params['initial_scenario'], turn_sequence=turn_sequence)

🚀 Starting D&D simulation...

=== D&D SIMULATION STARTING ===
Scenario: {'1': {'date': '2017-12-16T13:43:31', 'player': 'LightSpeed', 'character': 'Dungeon Master', 'in_combat': False, 'paragraphs': {'0': {'text': 'Guardians of Gridori', 'actions': ['name_mention: gridori'], 'label': 'in-character'}, '1': {'text': '[Recruitment Closed]', 'actions': [], 'label': 'in-character'}, '2': {'text': "General Information : A PbP 5E D&D campaign for 3-4 players, new players and veterans are welcome. New characters begin at first level. Race and class options are restricted only to the  Player's Handbook  (dragonborn are not available for this campaign). Ability scores can be determined by using standard array or point buy. Advancement will be achieved through milestones. If you are interested, please post a character concept including  name ,  race ,  class , and  background . Furthermore, please include a  brief  answer to the question:  Why are you a guardian?  Please consider the information 

## Export Results (Optional)

Save the simulation results for further analysis.

In [ ]:
# Export simulation results
import datetime
# Save to file
output_file = f"llm_simulation_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_file, 'w') as f:
    json.dump(game_session.game_log, f, indent=2)

print(f"💾 Results saved to: {output_file}")
print(f"📁 File size: {os.path.getsize(output_file)} bytes")

💾 Results saved to: llm_simulation_results_20250728_162818.json
📁 File size: 5903 bytes
